# Hackathon template

In [1]:
import xarray as xr

import qnt.data as qndata
import qnt.stepper as qnstepper
import qnt.stats as qnstats
import qnt.graph as qngraph

# hint 1: you can load a subset of the datasets, not all thousands of them.
# You can do this statically writing names for assets:

#asset = load_assets()
#asset_names = [x[["id"] for x in asset]
#asset_names

#data = qndata.load_data(min_date = "2015-01-01",
#                        dims     = ("time", "field", "asset"),
#                        assets   = ["NASDAQ:AAPL", "NASDAQ:AMZN"])

# or dynamically introducing a cross-sectional selection of assets according to traded volume;
# load all assets, and use a filter later:

data = qndata.load_data(min_date = "2015-01-01",
                        dims     = ("time", "field", "asset"))

prices = data.loc[::-1, "close", :]
volume = data.loc[::-1, "vol", :]

# a cross-sectional selection of assets according to traded volume:

value_volume = prices*volume
value_volume_min = value_volume.rolling({"time":250}).min()
volume_q = value_volume_min.quantile(0.9, {"asset"})

high_volume = (value_volume_min > volume_q).astype(int)
               
# define price returns according to different timescales (replace 1 with another integer)

prices_shifted = prices.shift({"time":1})
returns = (prices-prices_shifted)/prices_shifted

# define the volatility of returns:

vola_returns = returns.rolling({"time":250}).std()
vola_returns_q = vola_returns.quantile(0.5, {"asset"})

# define a trend indicator:

avg_prices = prices.rolling({"time":30}).mean()
avg_prices_l = prices.rolling({"time":100}).mean()

diff_rel = (avg_prices-avg_prices_l)/avg_prices_l

# hint 2: combine several conditions together to filter bad periods: here we trade
# liquid assets, introduce a trend, require high traded volume and low volatility:

is_liquid = data.loc[::-1, "is_liquid", :]
               
liq_avg_returns = (is_liquid > 0.0).astype(int) * \
                  (diff_rel > 0.0).astype(int) * \
                  high_volume * \
                  (vola_returns < vola_returns_q).astype(int)

weights = liq_avg_returns / abs(liq_avg_returns)
weights = weights.fillna(0.0)

stat = qnstats.calc_stat(data, weights, slippage_factor=0.05)

performance = stat.to_pandas()["equity"]
qngraph.make_plot_filled(performance.index, performance, name="PnL (equity)", type="log")

# hint 3: show rolling Sharpe ratio on a 3-year basis, and look at last value in the plot:
# if it is larger than 1, try to submit. This example is "almost" 1, tuning some parameter
# earlier or introducing some new indicator, or improving the asset selection leads to
# an in-sample Sharpe ratio larger than 1

SRchart = stat.to_pandas()["sharpe_ratio"].iloc[20:]
qngraph.make_plot_filled(SRchart.index, SRchart, color="#F442C5", name="Rolling SR")

qnstepper.write_output(weights)

fetched chunk 1/11 1s
fetched chunk 2/11 2s
fetched chunk 3/11 2s
fetched chunk 4/11 3s
fetched chunk 5/11 3s
fetched chunk 6/11 4s
fetched chunk 7/11 5s
fetched chunk 8/11 5s
fetched chunk 9/11 6s
fetched chunk 10/11 7s
fetched chunk 11/11 7s
Data loaded 8s


/usr/local/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1354: RuntimeWarning:

All-NaN slice encountered



write output: /home/username/fractions.nc.gz
